In [26]:
import pandas as pd
from datetime import datetime
from statistics import mean
import numpy as np
from NowcastingEco import NowcastingEco

In [27]:
# Local path for the headlines data
df = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/headlines_english_arabic_countries.csv')

/var/folders/jn/gt3868vd039by14gxmgb54b40000gn/T/ipykernel_99104/2683930366.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/headlines_english_arabic_countries.csv')


In [28]:
crash_test = NowcastingEco(df)
crash_test.clean_data() # Egypt/KSA/UAE

/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/CRP_Egyptian_Economy/NowcastingEco.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/CRP_Egyptian_Economy/NowcastingEco.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/CRP_Egyptian_Economy/NowcastingEco.py:101: SettingWithCopyWarning: 
A value is trying to be set o

,gkgrecordid,date,subsourcecommonname,tone,translationinfo,cleaned_xml,cleaned_locations,cleaned_url,cleaned_themes
2,20151011213000-1644,2015-10-11 21:30:00,competitor.com,"[4.83271375464684, 4.83271375464684, 0, 4.8327...",NaN,[NA],[United Arab Emirates],"[news, chaves, confirms, abu, dhabi, title, as...","[JOBS, SKILLS, AND, EDUCATION, JOBS, STRATEGIE..."
5,20151011150000-539,2015-10-11 15:00:00,yahoo.com,"[0.465116279069767, 0.930232558139535, 0.46511...",NaN,[NA],"[United Arab Emirates, United Arab Emirates, U...","[mideast, stocks, gulf, markets, edge, positiv...","[ECON, OILPRICE, ECON, STOCKMARKET, ECON, STOC..."
25,20151014104500-0,2015-10-14 10:45:00,khaleejtimes.com,"[-7.12589073634204, 0.475059382422803, 7.60095...",NaN,[NA],"[United Arab Emirates, United Arab Emirates, U...","[crime, man, murders, woman, gets, arrested, a...","[TAX, FNCACT, WOMAN, TAX, FNCACT, WOMAN, TAX, ..."
26,20151011080000-53,2015-10-11 08:00:00,skyscrapercity.com,"[1.27167630057803, 1.61849710982659, 0.3468208...",NaN,[NA],"[United Arab Emirates, United Arab Emirates, U...",[],[nan]
30,20151018111500-661,2015-10-18 11:15:00,zawya.com,"[9.58904109589041, 9.58904109589041, 0, 9.5890...",NaN,[NA],"[United Arab Emirates, United Arab Emirates, U...",[],"[TAX, FNCACT, ASSISTANT, TAX, FNCACT, DIRECTOR..."
...,...,...,...,...,...,...,...,...,...
773826,20230307023000-201,2023-03-07 02:30:00,nzcity.co.nz,"[2.79503105590062, 4.6583850931677, 1.86335403...",NaN,"[PUSSYCAT, DOLLS, SINGER, KIMBERLY, WYATT'S, F...","[United Arab Emirates, United Arab Emirates, U...",[news],"[GEN, HOLIDAY, GEN, HOLIDAY, DELAY, USPEC, UNC..."
773831,20230209123000-400,2023-02-09 12:30:00,witneygazette.co.uk,"[4.71698113207547, 5.18867924528302, 0.4716981...",NaN,"[THE, APPRENTICE, 2023:, REECE, DONNELLY, QUIT...","[United Arab Emirates, United Arab Emirates]","[reece, donnelly, quits, show, due, health, is...","[GENERAL, HEALTH, MEDICAL, TAX, FNCACT, CANDID..."
773839,20230307134500-702,2023-03-07 13:45:00,bignewsnetwork.com,"[7.73809523809524, 10.1190476190476, 2.3809523...",NaN,"[FEDERAL, TAX, AUTHORITY, LAUNCHES, 'TAX, 10',...",[United Arab Emirates],"[federal, tax, authority, launches, tax, digit...","[GENERAL, GOVERNMENT, GENERAL, GOVERNMENT, EPU..."
773840,20230307134500-1305,2023-03-07 13:45:00,khaleejtimes.com,"[0.352112676056338, 1.05633802816901, 0.704225...",NaN,"['IT, WAS, A, ROLLERCOASTER, RIDE, TO, SPACE',...","[United Arab Emirates, United Arab Emirates]","[uae, it, was, a, rollercoaster, ride, to, spa...","[MANMADE, DISASTER, IMPLIED, MANMADE, DISASTER..."


In [29]:
crash_test.tone_analysis(indicator=True) 

ERROR Invalid input
Invalid option. Please choose Egypt, UAE, or KSA.
Invalid option. Please choose Egypt, UAE, or KSA.
Invalid option. Please choose Egypt, UAE, or KSA.
Invalid option. Please choose Egypt, UAE, or KSA.
Invalid option. Please choose Egypt, UAE, or KSA.
Invalid option. Please choose Egypt, UAE, or KSA.
Invalid option. Please choose Egypt, UAE, or KSA.


FileNotFoundError: [Errno 2] No such file or directory: '/content/Bloomberg_Data_UAE.xlsx'